**NB : Ce code doit etre importé et exécuté sur Kaggle.com, il génére des erreurs sur les autres plateformes (comme Jupyter, Spider ...)**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/model-joblib/model_joblib
/kaggle/input/movielens/tags.csv
/kaggle/input/movielens/README.txt
/kaggle/input/movielens/ratings.csv
/kaggle/input/movielens/links.csv
/kaggle/input/movielens/movies.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv


In [2]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 48kB/s s eta 0:00:01     |█████████████████████████████▋  | 199.4MB 54.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 36.9MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216131250 sha256=8acbcb1704a1f311a494172295700662179e183d25bbd6c5d64a69c222012643
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:
import pandas as pd

In [6]:
#genome_scores = pd.read_csv("../input/movielens-20m-dataset/genome_scores.csv")
#genome_tags = pd.read_csv("../input/movielens-20m-dataset/genome_tags.csv")
#link = pd.read_csv("../input/movielens-20m-dataset/link.csv")
#movie = pd.read_csv("../input/movielens-20m-dataset/movie.csv")
#rating = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
#tag = pd.read_csv("../input/movielens-20m-dataset/tag.csv")

In [7]:
# La table des notes des films
movie_ratings = spark.read.csv("../input/movielens/ratings.csv", inferSchema=True, header=True)
movie_ratings = movie_ratings.select('userId', 'movieId', 'rating')

In [8]:
type(movie_ratings)

pyspark.sql.dataframe.DataFrame

In [9]:
# Cette contient environs 20 milions de lignes, on prend juste les 1 milions premiers
#movie_ratings = movie_ratings.head(1000000)

In [10]:
#type(movie_ratings)

In [11]:
# l'étape precedante nous retourne une liste, on crée une liste des noms des colonnes avec lesquels on vas structurer la liste movie_ratings
#columns = list(['userId', 'movieId', 'rating'])

In [12]:
# Création d'une dataframe à partir de la liste movie_ratings avec les noms de colonnes columns
#movie_ratings = spark.createDataFrame(movie_ratings, columns)

In [13]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows



In [14]:
movie_ratings.count()

1048575

In [15]:
# Alternating Least Squares (Alternance des moindres carrés)
from pyspark.ml.recommendation import ALS
# RegressionEvaluator pour évaluer la performance du modèle ALS
from pyspark.ml.evaluation import RegressionEvaluator
# CrossValidator pour diviser la dataset en training and testing
# ParamGridBuilder pour affiner les paramètres de notre modèle
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [16]:
# Création de training set et test set
(training, test) = movie_ratings.randomSplit([0.8, 0.2])

In [17]:
# Création du modèle ALS (Alternating Least Saqures)
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop', nonnegative=True)

# nonnegative=True : car on veut pas qu'il nous retourne des valeurs négatives

In [18]:
# Régler le modèle en utilisant ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(als.rank, [12, 13, 14]).addGrid(als.maxIter, [18, 19, 20]).addGrid(als.regParam, [.17, .18, .19]).build()

# On le donne :
# les paramètres des matrices U et P
# max iterations qui disent à Spark combien de fois alterner entre U et P pour minimiser l'erreur
# le paramètre de régularisation pour empêcher ALS de sur-adapter aux données (overfitting)

In [19]:
# Définir l'Évaluateur de régression, qui attend la prédiction des colonnes d'entrée
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# predictionCol='prediction' : le nom de la colonne des prédictions

In [20]:
# Construction de cross validation
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# estimator=als : pour utiliser le modèle de ALS

In [21]:
# Entrainer le modèle avec les données d'entraînement
model = cv.fit(training)

In [22]:
# Extraire le meilleur modèle de l'exercice de tournage à l'aide de ParamGridBuilder
best_model = model.bestModel

In [23]:
# Générer des prédictions et évaluer à l'aide de RMSE
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

# rmse : Écart quadratique moyen (Root-mean-square deviation)

In [24]:
# Afficher les métriques d'évaluation du modèle
print("RMSE = " + str(rmse))

RMSE = 0.8496662216339543


In [25]:
# Comparer les prédictions des évaluations des utilisateurs (ratings) avec les évaluations réels
predictions.sort('userId', 'rating').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   4133|   3.0| 2.5531554|
|     1|   4915|   3.0| 2.7755487|
|     1|   7164|   3.5|  3.436579|
|     1|   2947|   3.5|  3.641039|
|     1|   1348|   3.5|  3.530791|
|     1|   2804|   3.5| 3.8141332|
|     1|   1261|   3.5| 3.6215954|
|     1|   3265|   3.5| 3.6901639|
|     1|     29|   3.5|  3.663766|
|     1|   7387|   3.5| 3.6192462|
|     1|   7247|   3.5|  3.354847|
|     1|   3000|   3.5| 3.9022505|
|     1|   3476|   3.5| 3.4402213|
|     1|   8690|   3.5|  3.445186|
|     1|   7449|   3.5| 2.4115458|
|     1|   1994|   3.5| 3.4059916|
|     1|   1217|   3.5|  3.769691|
|     1|   3489|   4.0| 3.1725478|
|     1|   3479|   4.0|  3.515057|
|     1|   7454|   4.0|  2.950144|
+------+-------+------+----------+
only showing top 20 rows



In [26]:
# Génerer les prédictions des évaluations de tous les utilisateurs (Recommander 10 films pour chaque utilisateur)
users_recommendations = best_model.recommendForAllUsers(10)

In [27]:
users_recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|  1580|[[59295, 3.838721...|
|  4900|[[82931, 5.602125...|
|  5300|[[7758, 6.1101475...|
|  6620|[[7758, 5.903617]...|
|   471|[[7758, 5.2943616...|
|  1591|[[7758, 5.976639]...|
|  4101|[[7758, 5.343327]...|
|  1342|[[7758, 6.072916]...|
|  2122|[[26228, 4.701093...|
|  2142|[[727, 5.6486073]...|
|   463|[[7758, 5.8105145...|
|   833|[[95776, 5.425595...|
|  5803|[[7758, 6.549633]...|
|  3794|[[7758, 5.7973666...|
|  6654|[[5911, 6.5387616...|
|  1645|[[7758, 5.422957]...|
|  3175|[[95776, 6.524455...|
|  4935|[[82931, 4.783737...|
|   496|[[7758, 6.0197377...|
|  2366|[[7758, 5.8946195...|
+------+--------------------+
only showing top 20 rows



In [28]:
# SQLContext : Le point d'entrée pour travailler avec des données structurées (lignes et colonnes) dans Spark
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark)

In [29]:
# Ajout de la table des films pour joindre les noms des films avec la table de recommendations en résultat

In [30]:
movieDF = spark.read.csv("../input/movielens/movies.csv", inferSchema=True, header=True)

AnalysisException: 'Path does not exist: file:/kaggle/input/movielens/movie.csv;'

In [ ]:
movieDF.show()

In [ ]:
# Pour faciliter l'affichage de users_recommendations

def get_recs_for_user(recs):
    recs = recs.select("userId", "recommendations.movieId", "recommendations.rating")
    movies = recs.select("movieId").toPandas().iloc[:, 0].values
    ratings = recs.select("rating").toPandas().iloc[:, 0].values
    userIds = recs.select("userId").toPandas()
    ratings_matrix = pd.DataFrame(movies, columns=['movieId'])
    #ratings_matrix['userId'] = userIds
    ratings_matrix.insert(0, 'userId', userIds)
    ratings_matrix['ratings'] = ratings
    ratings_matrix_ps = sqlContext.createDataFrame(ratings_matrix)
    return ratings_matrix_ps

In [ ]:
users_recs = get_recs_for_user(users_recommendations)

In [ ]:
type(users_recs)

In [ ]:
users_recs.show()

In [ ]:
# Extraire de users_recommendations les recommandations pour un utilisateur spécifique
user_id = input("Donner l'id de l'utilisateur : ")
user_recs = users_recs.filter("userId="+user_id)

In [ ]:
user_recs.show()

In [ ]:
# Pour changer l'affichage de la liste des ids des films

z = []

for k,row in user_recs.toPandas().iterrows():
    for j in list(np.array(row.movieId).flat):
        z.append({'userId':row.userId, 'movieId':j})

user_recs = spark.createDataFrame(pd.DataFrame(z))

In [ ]:
# Joindre la dataframe des films recommandés pour l'utilisateur avec leurs titres et genres

user_recs = user_recs.join(movieDF, on='movieId')

In [ ]:
# Pour échanger les indexs de 'userId' et 'movieId'

user_recs = user_recs['userId', 'movieId', 'title', 'genres']

In [ ]:
# Affichage de la table de recommendation pour l'utilisateur demandé
user_recs.show()